In [3]:
import os
import sys
import glob
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm # Barre de progression pour Jupyter
import torch.nn as nn
import torch.optim as optim

# === CONFIGURATION ===
# Chemins relatifs (supposant que le notebook est à la racine du projet)
BASE_DIR = os.getcwd()

# Tes dossiers de données
IMAGES_DIR = os.path.join(BASE_DIR, "data", "dataset_continuous_lesions", "image")
MASKS_DIR = os.path.join(BASE_DIR, "data", "dataset_continuous_lesions", "mask")


# Tes dossiers de code et poids
SEGDINO_REPO = os.path.join(BASE_DIR, "SegDINO")
DINOV3_REPO = os.path.join(BASE_DIR, "dinov3")
WEIGHTS_PATH = os.path.join(BASE_DIR, "weights", "dinov3_vitl16_pretrain_lvd1689m-8aa4cbdd.pth")

# Dossier de sortie (Tu le crées toi-même ici !)
RESULTS_DIR = os.path.join(BASE_DIR, "results")
os.makedirs(RESULTS_DIR, exist_ok=True)

# Paramètres
#DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE = torch.device('cuda')
BATCH_SIZE = 8
EPOCHS = 30
LR = 2e-4

print(f"✅ Configuration chargée.")
print(f"📂 Résultats seront dans : {RESULTS_DIR}")
print(f"⚙️ Device : {DEVICE}")

# Vérification rapide
if not os.path.exists(SEGDINO_REPO):
    raise FileNotFoundError(f"Le dossier SegDINO est introuvable ici : {SEGDINO_REPO}")
if not os.path.exists(WEIGHTS_PATH):
    raise FileNotFoundError(f"Les poids sont introuvables ici : {WEIGHTS_PATH}")

✅ Configuration chargée.
📂 Résultats seront dans : /home/ulysse/Bureau/CD LAB/IRM/WORK/results
⚙️ Device : cpu


FileNotFoundError: Les poids sont introuvables ici : /home/ulysse/Bureau/CD LAB/IRM/WORK/weights/dinov3_vitl16_pretrain_lvd1689m-8aa4cbdd.pth

In [5]:
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

# Transformations (Standard pour DINO)
img_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

mask_transforms = transforms.Resize((224, 224), interpolation=transforms.InterpolationMode.NEAREST)

# === REMPLACE TA CELLULE DE DATASET PAR CELLE-CI ===

class PatientVolumetricDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        
        # Dictionnaire pour regrouper les fichiers
        # Clé = Patient_Axe_Sequence (ex: sub-strokecase0005_axial_seq02)
        # Valeur = Liste des fichiers
        self.volume_groups = {}
        
        all_files = sorted([f for f in os.listdir(img_dir) if f.endswith('.png')])
        
        # --- PARSING DU NOMMAGE SPÉCIFIQUE ---
        # Format : sub-strokecase0005_axial_019_seq02_01.png
        # parts[0] = sub-strokecase0005 (Patient)
        # parts[1] = axial              (Axe)
        # parts[2] = 019                (Slice Globale - on ignore pour le groupement)
        # parts[3] = seq02              (ID Séquence)
        # parts[4] = 01.png             (Position locale)
        
        for f in all_files:
            try:
                parts = f.split('_')
                if len(parts) >= 5:
                    # On construit la clé unique pour ce bloc 3D
                    # Clé = Patient + Axe + SeqID
                    unique_group_key = f"{parts[0]}_{parts[1]}_{parts[3]}"
                    
                    if unique_group_key not in self.volume_groups:
                        self.volume_groups[unique_group_key] = []
                    self.volume_groups[unique_group_key].append(f)
            except Exception as e:
                print(f"⚠️ Erreur parsing fichier {f}: {e}")
                
        self.group_ids = list(self.volume_groups.keys())
        
        # Stats pour vérifier
        print(f"✅ Dataset initialisé : {len(self.group_ids)} séquences continues trouvées.")
        if len(self.group_ids) > 0:
            example_key = self.group_ids[0]
            print(f"   Exemple de clé : {example_key}")
            print(f"   Contient {len(self.volume_groups[example_key])} images (profondeur).")

    def __len__(self):
        return len(self.group_ids)

    def __getitem__(self, idx):
        group_key = self.group_ids[idx]
        files = self.volume_groups[group_key]
        
        # TRI CRUCIAL : On doit trier les images pour qu'elles soient dans l'ordre (00, 01, 02...)
        # On trie sur la toute fin du fichier : le numéro avant .png
        # ex: ...seq02_01.png -> 1
        files.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))
        
        imgs = []
        masks = []
        
        for f_name in files:
            img_path = os.path.join(self.img_dir, f_name)
            mask_path = os.path.join(self.mask_dir, f_name)
            
            # --- Image ---
            img = Image.open(img_path).convert("RGB")
            if self.transform:
                img = self.transform(img)
            imgs.append(img)
            
            # --- Masque ---
            mask = Image.open(mask_path).convert("L")
            # Resize manuel en Nearest car les transforms sont souvent pour l'interpolation bilinéaire
            mask = mask.resize((224, 224), Image.NEAREST)
            mask_np = np.array(mask)
            mask_tensor = torch.from_numpy(mask_np > 0).float().unsqueeze(0)
            masks.append(mask_tensor)
        
        # Stack pour créer le volume
        # (Depth, 3, H, W)
        volume_img = torch.stack(imgs, dim=0)
        # (Depth, 1, H, W)
        volume_mask = torch.stack(masks, dim=0)
        
        return volume_img, volume_mask

# Création des Loaders (BATCH_SIZE DOIT ÊTRE 1 ici pour gérer des profondeurs variables)
full_ds = PatientVolumetricDataset(IMAGES_DIR, MASKS_DIR, transform=img_transforms)
train_size = int(0.8 * len(full_ds))
val_size = len(full_ds) - train_size
train_ds, val_ds = random_split(full_ds, [train_size, val_size], generator=torch.Generator().manual_seed(42))

# Important : batch_size=1
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False)

✅ Dataset initialisé : 687 séquences continues trouvées.
   Exemple de clé : sub-strokecase0001_axial_seq01
   Contient 2 images (profondeur).


In [3]:
# 1. Ajout du repo SegDINO au Path Python (pour trouver dpt.py et blocks.py)
if SEGDINO_REPO not in sys.path: sys.path.append(SEGDINO_REPO)

# 2. Importation de la classe DPT (c'est le vrai nom du modèle !)
try:
    from dpt import DPT
    print("✅ Classe DPT importée avec succès.")
except ImportError as e:
    print(f"❌ Erreur d'import : {e}")
    print(f"Vérifiez que le dossier {SEGDINO_REPO} contient bien dpt.py")
    raise e

# 3. Chargement du Backbone DINOv3 via torch.hub en LOCAL
# C'est la méthode utilisée dans train_segdino.py
print(f"🏗️ Chargement du backbone DINOv3 depuis : {DINOV3_REPO}")
print(f"💉 Poids : {WEIGHTS_PATH}")

try:
    # On charge le modèle 'small' (vits16)
    # source='local' force à utiliser le dossier dinov3 cloné au lieu d'internet
    backbone = torch.hub.load(
        repo_or_dir=DINOV3_REPO, 
        model='dinov3_vitl16', 
        source='local', 
        weights=WEIGHTS_PATH
    )
    print("✅ Backbone DINOv3 chargé.")
except Exception as e:
    print(f"❌ Erreur torch.hub : {e}")
    print("Vérifiez que le dossier 'dinov3' est bien le clone du repo facebookresearch/dinov3")
    raise e

# 4. Instanciation du modèle complet
# On passe le backbone chargé à la classe DPT
# nclass=1 car on fait de la segmentation binaire (Lésion vs Fond)
try:
    model = DPT(nclass=1, backbone=backbone)
    model = model.to(DEVICE)
    print("✅ Modèle complet (DPT + Backbone) prêt sur GPU/CPU.")
except Exception as e:
    print(f"❌ Erreur instanciation DPT : {e}")
    raise e

✅ Classe DPT importée avec succès.
🏗️ Chargement du backbone DINOv3 depuis : /home/maxime/Documents/CD Lab/IRM/SegDinov3/dinov3
💉 Poids : /home/maxime/Documents/CD Lab/IRM/SegDinov3/weights/dinov3_vitl16_pretrain_lvd1689m-8aa4cbdd.pth


/home/maxime/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Backbone DINOv3 chargé.
✅ Modèle complet (DPT + Backbone) prêt sur GPU/CPU.


In [4]:
class DiceLoss(nn.Module):
    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, inputs, targets):
        # inputs: logits (sortie du modèle)
        # targets: labels (0 ou 1)
        
        # Sigmoid pour avoir des probas entre 0 et 1
        inputs = torch.sigmoid(inputs)
        
        # Aplatir les dimensions (Batch, Depth, H, W -> Vecteur)
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()
        dice = (2. * intersection + self.smooth) / (inputs.sum() + targets.sum() + self.smooth)
        
        # On veut maximiser le Dice, donc minimiser (1 - Dice)
        return 1 - dice

# On combine BCE et Dice pour la stabilité
class CombinedLoss(nn.Module):
    def __init__(self):
        super(CombinedLoss, self).__init__()
        self.bce = nn.BCEWithLogitsLoss()
        self.dice = DiceLoss()
        
    def forward(self, inputs, targets):
        return 0.5 * self.bce(inputs, targets) + 0.5 * self.dice(inputs, targets)

In [5]:
class SegDINO_3D_Wrapper(nn.Module):
    def __init__(self, model_2d):
        super().__init__()
        self.model_2d = model_2d
        
        # On débloque les gradients 2D (sera géré par l'optimiseur)
        for param in self.model_2d.parameters():
            param.requires_grad = True
            
        # --- CORRECTION ICI : InstanceNorm3d au lieu de BatchNorm3d ---
        # C'est beaucoup plus stable quand batch_size = 1
        self.refinement_3d = nn.Sequential(
            nn.Conv3d(1, 16, kernel_size=3, padding=1),
            nn.InstanceNorm3d(16),  # <--- CHANGEMENT CRITIQUE
            nn.ReLU(inplace=True),
            
            nn.Conv3d(16, 16, kernel_size=3, padding=1),
            nn.InstanceNorm3d(16),  # <--- CHANGEMENT CRITIQUE
            nn.ReLU(inplace=True),
            
            nn.Conv3d(16, 1, kernel_size=1)
        )
        
        # Init à zéro pour connexion résiduelle douce
        nn.init.zeros_(self.refinement_3d[-1].weight)
        nn.init.zeros_(self.refinement_3d[-1].bias)

    def forward(self, volume_img):
        b, depth, c, h, w = volume_img.shape
        
        # 1. Passage 2D
        img_flat = volume_img.view(b * depth, c, h, w)
        out = self.model_2d(img_flat)
        
        if isinstance(out, dict): pred_2d = out['pred']
        elif isinstance(out, (list, tuple)): pred_2d = out[0]
        else: pred_2d = out
            
        # 2. Reshape 3D
        x_3d_input = pred_2d.view(b, depth, 1, h, w).permute(0, 2, 1, 3, 4)
        
        # 3. Passage 3D Résiduel
        residual = self.refinement_3d(x_3d_input)
        
        # Connexion résiduelle : Le 3D corrige le 2D
        output = x_3d_input + residual
        
        return output.permute(0, 2, 1, 3, 4)

In [6]:
# --- LOSS PLUS DOUCE ---
class WeightedCombinedLoss(nn.Module):
    def __init__(self, weight_bce=20.0): # On réduit à 20 (suffisant pour le déséquilibre)
        super(WeightedCombinedLoss, self).__init__()
        self.bce = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([weight_bce]).to(DEVICE))
        self.dice_loss = DiceLoss(smooth=1.0)
        
    def forward(self, inputs, targets):
        return 0.5 * self.bce(inputs, targets) + 0.5 * self.dice_loss(inputs, targets)

In [7]:
def compute_dice_score(logits, targets, smooth=1e-6):
    """
    Calcule le Dice Score pour un volume ou une image.
    logits : Sortie brute du réseau (avant Sigmoid)
    targets : Masque binaire réel (0 ou 1)
    smooth : Petit nombre pour éviter la division par 0 si tout est noir
    """
    # 1. On applique Sigmoid pour avoir des probabilités entre 0 et 1
    probs = torch.sigmoid(logits)
    
    # 2. On seuille à 0.5 pour avoir du binaire (0 ou 1)
    preds = (probs > 0.5).float()
    
    # 3. Aplatir les tenseurs (transformer le volume 3D en une longue ligne)
    # Cela permet de calculer l'intersection facilement peu importe les dimensions
    preds_flat = preds.view(-1)
    targets_flat = targets.view(-1)
    
    # 4. Calcul de l'intersection et de l'union
    intersection = (preds_flat * targets_flat).sum()
    union = preds_flat.sum() + targets_flat.sum()
    
    # 5. Formule du Dice : 2 * Intersection / (Total pixels prédits + Total pixels réels)
    dice = (2. * intersection + smooth) / (union + smooth)
    
    return dice.item()

In [ ]:
from tqdm import tqdm # Barre de progression pour Jupyter
import torch.nn as nn
import torch.optim as optim

# Listes pour stocker l'historique (pour les graphiques plus tard)
history = {'train_loss': [], 'train_dice': [], 'val_dice': []}
best_val_dice = 0.0

# ==========================================
# CONFIGURATION FINALE ET STABLE
# ==========================================

# 1. Réinitialisation propre du modèle
model_3d = SegDINO_3D_Wrapper(model).to(DEVICE)

# 2. On gèle le backbone DINO (trop lourd), on entraîne le reste
for param in model_3d.model_2d.backbone.parameters():
    param.requires_grad = False

# 3. Optimiseur (LR ajusté pour l'entraînement complet)
optimizer = optim.AdamW(
    filter(lambda p: p.requires_grad, model_3d.parameters()), 
    lr=2e-4, 
    weight_decay=1e-4
)

# 4. Loss Pondérée (Stable)
criterion = WeightedCombinedLoss(weight_bce=10.0)

# ==========================================
# BOUCLE D'ENTRAÎNEMENT
# ==========================================

history = {'train_loss': [], 'train_dice': [], 'val_dice': []}
best_val_dice = 0.0

print(f"🚀 Démarrage Entraînement Final (Mode Stable)")
print(f"   Note: Le Dice va commencer bas et monter progressivement.")

# Import pour le Mixed Precision
from torch.cuda.amp import autocast, GradScaler

# 1. Création du Scaler (avant la boucle)
scaler = GradScaler()

print(f"🚀 Démarrage Entraînement Final (Mode AMP - Économie Mémoire)")

for epoch in range(EPOCHS):
    model_3d.train()
    running_loss = 0.0
    running_dice = 0.0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
    
    for vol_img, vol_mask in pbar:
        vol_img, vol_mask = vol_img.to(DEVICE), vol_mask.to(DEVICE)
        
        optimizer.zero_grad()
        
        # --- CHANGEMENT 1 : Context Autocast ---
        # PyTorch gère auto le passage en float16 pour ce qui est compatible
        with autocast():
            outputs_3d = model_3d(vol_img)
            loss = criterion(outputs_3d, vol_mask)
        
        # --- CHANGEMENT 2 : Backprop via Scaler ---
        # Le scaler gère les gradients pour éviter qu'ils ne soient trop petits (underflow)
        scaler.scale(loss).backward()
        
        # On doit "unscale" avant de clipper les gradients
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model_3d.parameters(), max_norm=1.0)
        
        scaler.step(optimizer)
        scaler.update()
        
        # Ménage immédiat (Optionnel mais aide si tu es limite)
        # del vol_img, vol_mask, outputs_3d
        # torch.cuda.empty_cache() 
        
        # Métriques (Attention, on repasse en float32 pour le calcul CPU)
        dice = compute_dice_score(outputs_3d.detach().float(), vol_mask.float())
        running_loss += loss.item()
        running_dice += dice
        
        pbar.set_postfix({'loss': f"{loss.item():.3f}", 'dice': f"{dice:.3f}"})
        
    # ... (Le reste de la validation est inchangé, mets juste 'with autocast():' dans la validation aussi)
        
    avg_train_loss = running_loss / len(train_loader)
    avg_train_dice = running_dice / len(train_loader)
    
    # --- VALIDATION ---
    model_3d.eval()
    val_dice_total = 0.0
    
    with torch.no_grad():
        for vol_img, vol_mask in val_loader:
            vol_img, vol_mask = vol_img.to(DEVICE), vol_mask.to(DEVICE)
            outputs_3d = model_3d(vol_img)
            val_dice_total += compute_dice_score(outputs_3d, vol_mask)
            
    avg_val_dice = val_dice_total / len(val_loader)
    
    # --- LOGS & SAUVEGARDE ---
    history['train_loss'].append(avg_train_loss)
    history['train_dice'].append(avg_train_dice)
    history['val_dice'].append(avg_val_dice)
    
    print(f"End Epoch {epoch+1} | Train Dice: {avg_train_dice:.4f} | Val Dice: {avg_val_dice:.4f}")
    
    # Sauvegarde au moindre progrès sur la validation
    if avg_val_dice > best_val_dice:
        best_val_dice = avg_val_dice
        torch.save(model_3d.state_dict(), os.path.join(RESULTS_DIR, "best_segdino_3d.pth"))
        print(f"   💾 Modèle sauvegardé (Nouveau record validation)")

print("🏁 Terminé !")

/tmp/ipykernel_17507/914668173.py:44: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


🚀 Démarrage Entraînement Final (Mode Stable)
   Note: Le Dice va commencer bas et monter progressivement.
🚀 Démarrage Entraînement Final (Mode AMP - Économie Mémoire)


Epoch 1/30:   0%|                                       | 0/600 [00:00<?, ?it/s]/tmp/ipykernel_17507/914668173.py:62: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1/30: 100%|█████| 600/600 [05:19<00:00,  1.88it/s, loss=0.382, dice=0.368]


End Epoch 1 | Train Dice: 0.3044 | Val Dice: 0.4131
   💾 Modèle sauvegardé (Nouveau record validation)


Epoch 2/30: 100%|█████| 600/600 [05:06<00:00,  1.96it/s, loss=3.815, dice=0.397]


...

Epoch 9/30: 100%|█████| 600/600 [05:22<00:00,  1.86it/s, loss=0.362, dice=0.423]


End Epoch 9 | Train Dice: 0.5727 | Val Dice: 0.5523
   💾 Modèle sauvegardé (Nouveau record validation)


Epoch 10/30: 100%|████| 600/600 [05:20<00:00,  1.87it/s, loss=0.316, dice=0.428]


End Epoch 10 | Train Dice: 0.5836 | Val Dice: 0.5340


Epoch 11/30:  96%|███▊| 578/600 [05:10<00:11,  1.86it/s, loss=0.489, dice=0.370]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# ==========================================
# 1. AFFICHAGE DES COURBES D'APPRENTISSAGE
# ==========================================

plt.figure(figsize=(15, 5))

# Courbe de Loss
plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Train Loss', color='red', marker='o')
plt.title('Évolution de la Loss (Erreur)')
plt.xlabel('Époques')
plt.ylabel('BCE Loss')
plt.grid(True, alpha=0.3)
plt.legend()

# Courbe de Dice
plt.subplot(1, 2, 2)
plt.plot(history['train_dice'], label='Train Dice', color='blue', linestyle='--')
plt.plot(history['val_dice'], label='Validation Dice', color='green', marker='o', linewidth=2)
plt.title('Évolution du Dice Score (Qualité Segmentation)')
plt.xlabel('Époques')
plt.ylabel('Dice Score (0 à 1)')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "learning_curves.png"))
plt.show()

# ==========================================
# 2. VISUALISATION DES PRÉDICTIONS 3D
# ==========================================
print("\n📸 Génération d'un exemple de validation...")

model_3d.eval()

# On récupère UN patient complet dans le loader de validation
vol_img, vol_mask = next(iter(val_loader)) 

# Envoi sur GPU et Prédiction
vol_img = vol_img.to(DEVICE)
with torch.no_grad():
    logits_3d = model_3d(vol_img)
    preds_3d = (torch.sigmoid(logits_3d) > 0.5).float()

# --- PRÉPARATION POUR AFFICHAGE ---
depth = vol_img.shape[1] # Dimension 1 est la profondeur

# On choisit 3 slices réparties (ex: 25%, 50%, 75%)
# On s'assure que les indices sont valides (min 0, max depth-1)
slice_indices = [int(depth*0.25), int(depth*0.50), int(depth*0.75)]
slice_indices = [min(i, depth-1) for i in slice_indices]

fig, axes = plt.subplots(len(slice_indices), 3, figsize=(12, 4 * len(slice_indices)))
plt.suptitle(f"Résultats sur un patient (Profondeur totale: {depth} slices)", fontsize=16)

# Gestion du cas où il n'y a qu'une seule slice à afficher (si depth est très petit)
if len(slice_indices) == 1: axes = np.expand_dims(axes, axis=0)

for i, slice_idx in enumerate(slice_indices):
    # --- 1. Image IRM (Input) ---
    # Shape: (Batch, Depth, Channels, H, W) -> On prend [0, slice_idx]
    img_slice = vol_img[0, slice_idx].cpu().permute(1, 2, 0).numpy()
    
    # Denormalization pour affichage propre
    img_min, img_max = img_slice.min(), img_slice.max()
    if img_max > img_min:
        img_slice = (img_slice - img_min) / (img_max - img_min)
    
    # --- 2. Vérité Terrain ---
    # Shape: (Batch, Depth, Channel, H, W) -> On prend [0, slice_idx, 0]
    mask_slice = vol_mask[0, slice_idx, 0].cpu().numpy()
    
    # --- 3. Prédiction (CORRECTION ICI) ---
    # Shape: (Batch, Depth, Channel, H, W) -> On prend [0, slice_idx, 0]
    pred_slice = preds_3d[0, slice_idx, 0].cpu().numpy()
    
    # --- AFFICHAGE ---
    # Colonne 1 : IRM
    axes[i, 0].imshow(img_slice)
    axes[i, 0].set_title(f"Slice {slice_idx} - IRM", fontsize=10)
    axes[i, 0].axis('off')
    
    # Colonne 2 : Masque Réel
    axes[i, 1].imshow(mask_slice, cmap='gray')
    axes[i, 1].set_title(f"Slice {slice_idx} - Vérité", fontsize=10)
    axes[i, 1].axis('off')
    
    # Colonne 3 : Prédiction
    axes[i, 2].imshow(pred_slice, cmap='gray')
    
    # Petit calcul de Dice local pour le titre
    inter = (pred_slice * mask_slice).sum()
    dice_slice = (2. * inter) / (pred_slice.sum() + mask_slice.sum() + 1e-6)
    
    col = 'green' if dice_slice > 0.7 else 'red'
    axes[i, 2].set_title(f"Pred (Dice: {dice_slice:.2f})", color=col, fontweight='bold', fontsize=10)
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# === CELLULE DE DEBUG ===
print("🔍 INSPECTION DES DONNÉES")

# On prend un batch (un patient)
vol_img, vol_mask = next(iter(train_loader))

print(f"Format Image : {vol_img.shape}")
print(f"Format Masque : {vol_mask.shape}")

# 1. Vérifier les valeurs de l'image (doit être ~ entre -2 et +2 après normalisation)
print(f"\n--- IMAGE ---")
print(f"Min: {vol_img.min():.4f}, Max: {vol_img.max():.4f}, Mean: {vol_img.mean():.4f}")
if vol_img.max() > 10 or vol_img.min() < -10:
    print("⚠️ ALERTE : Les valeurs de l'image semblent anormales pour DINO (Normalisation ?)")

# 2. Vérifier le Masque (DOIT être 0.0 et 1.0 uniquement)
print(f"\n--- MASQUE ---")
unique_vals = torch.unique(vol_mask)
print(f"Valeurs uniques dans le masque : {unique_vals}")
print(f"Nombre de pixels lésion (1.0) : {vol_mask.sum().item()}")

if len(unique_vals) > 2:
    print("⚠️ ALERTE : Le masque n'est pas binaire ! Il contient d'autres valeurs.")
if vol_mask.sum() == 0:
    print("⚠️ ALERTE : Ce patient n'a AUCUNE lésion (masque vide). Le modèle ne peut rien apprendre ici.")

In [ ]:
from tqdm import tqdm # Barre de progression pour Jupyter
import torch.nn as nn
import torch.optim as optim

print("🧪 TEST D'OVERFITTING SUR UN SEUL PATIENT")

# On prend UN SEUL patient et on le fige
single_img, single_mask = next(iter(train_loader))
single_img, single_mask = single_img.to(DEVICE), single_mask.to(DEVICE)

'''
# On réinitialise un petit modèle pour le test
test_optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model_3d.parameters()), lr=1e-3)
test_criterion = CombinedLoss() # Ta loss actuelle
'''

# Réinstancie le modèle (pour effacer l'historique du crash)
model_3d = SegDINO_3D_Wrapper(model).to(DEVICE)

# On gèle le backbone DINO
for param in model_3d.model_2d.backbone.parameters():
    param.requires_grad = False

# Learning Rate un peu plus doux
test_optimizer = optim.AdamW(
    filter(lambda p: p.requires_grad, model_3d.parameters()), 
    lr=1e-4, 
    weight_decay=0 # Pas de decay pour le test d'overfitting
)

# Loss pondérée modérée
test_criterion = WeightedCombinedLoss(weight_bce=10.0) # 10 au lieu de 20 ou 100

model_3d.train()

pbar = tqdm(range(50), desc="Overfitting Test")
losses = []
dices = []

for i in pbar:
    test_optimizer.zero_grad()
    
    output = model_3d(single_img)
    loss = test_criterion(output, single_mask)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model_3d.parameters(), max_norm=1.0)
    test_optimizer.step()
    
    # Dice
    dice = compute_dice_score(output.detach(), single_mask)
    
    losses.append(loss.item())
    dices.append(dice)
    pbar.set_postfix({'loss': loss.item(), 'dice': dice})

# Affichage résultat
import matplotlib.pyplot as plt
plt.plot(dices)
plt.title("Dice Score sur 1 patient (Doit atteindre ~1.0)")
plt.show()

if dices[-1] > 0.8:
    print("✅ LE MODÈLE FONCTIONNE ! Il est capable d'apprendre.")
    print("   -> Le problème vient donc de la difficulté des données ou de la variété.")
else:
    print("❌ ECHEC : Le modèle n'arrive même pas à apprendre par cœur 1 image.")
    print("   -> Problème d'architecture, de Loss, ou de Learning Rate.")